In [27]:
from mangaki.algo.dataset import Dataset
from mangaki.utils.values import rating_values

queryset = Rating.objects.all()  #.exclude(choice__in=['willsee', 'wontsee']) # Security if the dataset is public
triplets = queryset.values_list('user_id', 'work_id', 'choice')
titles = Work.objects.order_by('id').values_list('id', 'title')
categories = Work.objects.order_by('id').values_list('id', 'category__slug')
dataset = Dataset()

In [28]:
anonymized = dataset.make_anonymous_data(triplets, convert=lambda choice: rating_values[choice], ordered=True)

In [29]:
dataset.anonymized.y

array([ 0.5, -0.5,  4. , ..., -0.5,  0.1,  2. ])

In [25]:
dataset.titles = dict(titles)
dataset.categories = dict(categories)

In [26]:
dataset.save_csv(suffix='-ordered')

In [5]:
dataset.save('balse.pickle')